In [ ]:
import abscr
from abscr.omero_connection import connector
from abscr.segmentation import segmentor
from abscr.analysis import counter

### Segmentation of an image from Omero.

Connect to Omero with your username and type in your password.

In [ ]:
user = ''
host = ''
client = connector.OmeroClient(username=user, host=host)

In [ ]:
client.show_user_summary()

In [ ]:
# Set your group id
group_id = 611  # ABSCR group
client.set_omero_group(group_id)

# You can also find id of particular image in sanger omero webclient
image_id = 41440
img_obj = client.get_image_cursor(image_id=image_id)

x, y = 60000, 17000
size = (2000, 2000)

img_region = client.get_image_jpg_region(img_obj, x, y, size)

img_region

In [ ]:
client.close()

Segmentor's method `predict_all` is designed to segment both epithelial and immune cells on buccal swab slides. 

By default, it produces a `.txt` file with masks and a `.png` file with those masks plotted on the original image. The output files are saved in the `savedir` directory (which is the current working directory by default). If the specified `savedir` directory does not exist it will be created. You can pass either a full or relative path as the `savedir`. Setting the `plot_segm` parameter to `True` will plot the masks in the notebook (or script).

If the image to segment is a region of the bigger image from Omero, then the output `.png` and `.txt` files will have `_x_y_wxh` suffix in their filenames. Here `(x, y)` are the top left coordinates of the region, and `w` and `h` are the width and height of the region respectively. 

The return value of the `predict_all` method is a `BuccalSwabSegmentation` class object with masks, flows, styles and diams for each cell type. (See usage examples in the Cell counter usage section).

**Note**: You can run the method without specifying the parameters, though this may result in a poor segmentation result. This highly depends on specifying a proper cell diameter when running a model. In turn, the cell diameter parameter depends on the image scaling (i.e. the distance from the microscope to the slide). The default cell diameter value in pixels is 100.

In [ ]:
segmentor = segmentor.Segmentor()
result = segmentor.predict_all(img_region, plot_segm=True, save_png=False, diameter_epithelial=65)

There are methods to segment only epithelial or immune cells. (Though immune cell segmentation is yet to be implemented).

E.g. the `predict_epithelial` method uses a cellpose model to segment epithelial cells. The default parameters are:
- `diameter` = 30
- `flow_threshold` = 0.4
- `cellprob_threshold` = 0.0
- `channels` = [0, 0]
- `invert` = True
- `model_type` = cyto

These parameters can be modified if you think this will improve the segmentation accuracy.

You can use the `plot_segmentation` method to plot the segmentation result from an image and an array of masks. To save the plots produced by this method you can use the `save_png=True` option. The saving directory for the output `.png` file can be specified with the `savedir` parameter.

In [ ]:
epithelial_segmentation = segmentor.predict_epithelial(img_region, diameter=65)

# this plots the original image and epithelial masks
segmentor.plot_segmentation(img_region, [epithelial_segmentation.masks]) 

# this additionaly plots cellpose flows (just as an example)
segmentor.plot_segmentation(img_region, [epithelial_segmentation.masks, epithelial_segmentation.flows[0]])

### Segmentation of an image from the local storage.

You can also pass a filename, a path, or a file object to the `predict_all` method.

**Note**: if you want to segment an image from a different location other than the current one the full path may be needed. The full path must start with `/`.

In the following example, I manually uploaded a region from the `020_Buccal_05.04.2022` slide in the directory where the current notebook is located. After running the `predict_all` method a new directory named `segmentation_result/` will be created in the current one and the output files will be saved there.

In [ ]:
image_name = '020_Buccal_05.04.2022_small.jpeg' 
result2 = segmentor.predict_all(image_name, plot_segm=True, savedir='segmentation_results/', diameter_epithelial=100)

### Cell counter usage

The library has a counter class which has the `count_cells_from_masks` method. The method counts cells from the masks obtained after the segmentation step.

This method accepts any of the following inputs:
- a `.txt` file with masks
- an `.npy` file with masks
- an `np.ndarray` object with masks

In [ ]:
cell_counter = counter.CellCounter()

You can pass multiple sets of masks to the method to get a tuple of cell counts.

In [ ]:
print(cell_counter.count_cells_from_masks(epithelial_segmentation.masks))
cell_counter.count_cells_from_masks(epithelial_segmentation.masks, result2.epithelial_masks)